<h1>sign laungauge interpertation</h1>



<h3>Import libraryes</h3>

In [2]:
import tensorflow as tf
import cv2
import matplotlib as plt


import numpy as np
import time


import mediapipe as mp

<h3>Recording frames & getting Landmarks</h3>


In [3]:
mp_pos=mp.solutions.pose
mp_draw=mp.solutions.drawing_utils

In [4]:
def draw(frame,landmarks,mp_draw,mp_hol):
        
        mp_draw.draw_landmarks(frame,landmarks.pose_landmarks,mp_hol.POSE_CONNECTIONS,
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1))

In [4]:
#just capture
cam=cv2.VideoCapture(0)

with mp_pos.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hol:
    while cam.isOpened():
        ret,frame=cam.read()
        if not ret:
            break
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        landmarks=hol.process(image)
           
        #drawing on image
        draw(frame,landmarks,mp_draw,mp_pos)
        
        frame=cv2.flip(frame,1)
        cv2.imshow("capture",frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cam.release()
    cv2.destroyAllWindows()


<h3>Extracting landmarks</h3>

In [17]:

def extract_landmarks(landmarks):
    if landmarks.pose_landmarks:
        pose=np.array([[p.x,p.y,p.z,p.visibility] for p in landmarks.pose_landmarks.landmark]).flatten()
    else:
        pose=np.zeros(132,)
    

    return np.concatenate([pose])
        

In [32]:
print("pse shape: ",pose.shape,"r hand shape: ",right_hand.shape,"l hand shape: ",left_hand.shape,"face shape: ",face.shape)
extract_landmarks(landmarks).shape

pse shape:  (132,) r hand shape:  (63,) l hand shape:  (63,) face shape:  (1404,)


(1662,)

### collecting data

<h4>Making folders</h4>

In [21]:
import os

In [22]:
path_=os.path.join("Data")
no_frames=30
no_video=30
actions=["cheating","not cheating"]


In [23]:
for action in actions:
    for vid in range(no_video):
        try:
            os.makedirs(os.path.join(path_,action,str(vid)))
        except:
            pass

<h4>collecting frames</h4>

In [25]:
cam=cv2.VideoCapture(0)
flag=False
set=True
with mp_pos.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hol:
    for aindx,action in enumerate(actions):
        if flag:
                break
        for vid in range(no_video):
            if flag:
                break
            for frame_no in range(no_frames+1):
                
            
                ret,frame=cam.read()
                if not ret:
                    break
                image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                landmarks=hol.process(image)
                   
                #drawing on image
                draw(frame,landmarks,mp_draw,mp_pos)
                frame=cv2.flip(frame,1)
                
                cv2.putText(frame,"collecting frame for {} video no {} Frame no {}".format(action,vid,frame_no),(120,20),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                
                if aindx==1 and set:
                    set=False
                    cv2.putText(frame,"Starting collection for not cheating",(0,50),
                                cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),4,cv2.LINE_AA)
                    cv2.putText(frame,"collecting frame for {} video no {} Frame no {}".format(action,vid,frame_no),(120,20),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow("capture",frame)
                    if cv2.waitKey(50000) or 0xFF == ord('c'): #change capture technique
                        continue




                
                feature=extract_landmarks(landmarks)
                path=os.path.join(path_,action,str(vid),str(frame_no-1))
                np.save(path,feature)

                
                    
                cv2.imshow("capture",frame)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    flag=True
                    break
        
    cam.release()
    cv2.destroyAllWindows()


In [24]:
cam.release()
cv2.destroyAllWindows()

<h3>Loading dataset</h3>

In [26]:
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [27]:
label_map=[name for name in os.listdir(path_)] #getting label names

print(label_map)

['cheating', 'not cheating']


In [28]:
data=[]
labels=[]
for i,label in enumerate(label_map):
    for vid_no in range(no_video):
        vid=[]
        for frame_no in range(0,no_frames):
            res=np.load(os.path.join(path_,label,str(vid_no),str(frame_no)+".npy"))
            vid.append(res)
        data.append(vid)
        labels.append(i)
            
            

In [16]:
print(data)

[[array(False), array(False), array(False), array(False), array(False), array(False), array(False), array(False), array(False), array(False), array(False), array([ 0.54428422,  0.2954078 , -0.37138936,  0.99970102,  0.53737044,
        0.29808384, -0.38401228,  0.99986231,  0.53560042,  0.29544583,
       -0.38403803,  0.9998486 ,  0.53378034,  0.29282519, -0.38418683,
        0.99987125,  0.54225981,  0.30604771, -0.38970858,  0.99972707,
        0.54433918,  0.30852953, -0.3897121 ,  0.9996165 ,  0.54628885,
        0.31096235, -0.38977462,  0.99973053,  0.53199208,  0.28069833,
       -0.33599117,  0.99991369,  0.5467    ,  0.30915576, -0.36464995,
        0.99984109,  0.54264116,  0.28127009, -0.34374559,  0.99965847,
        0.5493018 ,  0.28973669, -0.35252607,  0.99977452,  0.51923454,
        0.25405839, -0.23423526,  0.99913329,  0.53695756,  0.26761165,
       -0.2685442 ,  0.99531895,  0.54445386,  0.24461953, -0.15418874,
        0.90758973,  0.543486  ,  0.28208688, -0.179

In [29]:
data=np.array(data)
labels=np.array(labels)

In [30]:
print(data.shape)
print(len(labels))


(60, 30, 132)
60


In [31]:
labels=to_categorical(np.array(labels)).astype(int)

In [32]:
labels.shape

(60, 2)

In [33]:
xtrain,xtest,ytrain,ytest=train_test_split(data,labels,test_size=0.05)

In [34]:
ytest.shape

(3, 2)

<h3>Model Architecture</h3>

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [36]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [44]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,132)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))

In [47]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [39]:
model.fit(xtrain, ytrain, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
2/2 [==============================] - 5s 176ms/step - loss: 0.6860 - categorical_accuracy: 0.5263
Epoch 2/2000
2/2 [==============================] - 0s 139ms/step - loss: 0.6227 - categorical_accuracy: 0.5965
Epoch 3/2000
2/2 [==============================] - 0s 142ms/step - loss: 0.6724 - categorical_accuracy: 0.6491
Epoch 4/2000
2/2 [==============================] - 0s 142ms/step - loss: 0.7330 - categorical_accuracy: 0.5088
Epoch 5/2000
2/2 [==============================] - 0s 141ms/step - loss: 0.6993 - categorical_accuracy: 0.5088
Epoch 6/2000
2/2 [==============================] - 0s 140ms/step - loss: 0.6672 - categorical_accuracy: 0.5614
Epoch 7/2000
2/2 [==============================] - 0s 192ms/step - loss: 0.6472 - categorical_accuracy: 0.6667
Epoch 8/2000
2/2 [==============================] - 0s 152ms/step - loss: 0.6295 - categorical_accuracy: 0.5614
Epoch 9/2000
2/2 [==============================] - 0s 147ms/step - loss: 0.5919 - categorical_accuracy:

KeyboardInterrupt: 

In [48]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            50432     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 204,962
Trainable params: 204,962
Non-tr

<h3>saving</h3>

In [41]:
model.save(os.path.join("models","main1.h5"))

In [42]:
del model 

In [50]:
model.load_weights(os.path.join("models","main1.h5"))

<h3>Evaluation</h3>

In [51]:

np.array([xtest[0]]).shape

(1, 30, 132)

In [52]:
res=model.predict(xtest)

1/1 [==============================] - 0s 315ms/step


In [53]:
ytrue = np.argmax(ytest, axis=1).tolist()
res = np.argmax(res, axis=1).tolist()

In [54]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [55]:
multilabel_confusion_matrix(ytrue, res)

array([[[2, 1],
        [0, 0]],

       [[0, 0],
        [1, 2]]], dtype=int64)

In [56]:
accuracy_score(ytrue,res)

0.6666666666666666

<h3>REALTIME TESTING</h3>

In [ ]:
#just capture
cam=cv2.VideoCapture(0)
action=[]
label_map=["not cheating","cheating"]
text=" "
# predictions = []

trsh=0.9
res=np.array([0,0])

with mp_pos.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hol:
    while cam.isOpened():
        ret,frame=cam.read()
        if not ret:
            break
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        landmarks=hol.process(image)

        points=extract_landmarks(landmarks)

        #getting 30 frames of action
        action.append(points)
        action=action[-30:]
        if len(action)>=30:
            res=model.predict(np.expand_dims(action,axis=0))[0]
            # predictions.append(np.argmax(res))
            p_idx=np.argmax(res)
            # predictions=predictions[-10:]
            # print(predictions)
            # if np.unique(predictions)[-1]==np.argmax(res): 
               
            if  res[p_idx]>trsh:                             
                text=label_map[p_idx]
            

                
            
                
                
              
           
        #drawing on image
        draw(frame,landmarks,mp_draw,mp_pos)
        frame=cv2.flip(frame,1)

        cv2.rectangle(frame, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(frame,text, (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow("capture",frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cam.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 0s 39ms/step


In [10]:
cam.release()
cv2.destroyAllWindows()
del res

NameError: name 'cam' is not defined

In [13]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
